# Librairies

In [1]:
import geopandas as gpd
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pylab as plt

# Read data

In [2]:
df = pd.read_excel('raw/DTV_DTVw_Download.xlsx', skiprows=4)

In [3]:
df['Zählstelle'].unique()

array([1013, 1017, 1018, 1022, 1035, 1041, 1043, 1045, 1054, 1061, 1077,
       1081, 1097, 1102, 1114, 1123, 1134, 1145, 1154, 1164, 1173, 1188,
       1196, 1204, 1205, 1208, 1212, 1221, 2018, 2028, 2031, 2042, 2056,
       2065, 2076, 2082, 2091, 2104, 2111, 2113, 2115, 2118, 2121, 2123,
       2125, 2127, 2134, 2141, 2152, 2163, 2172, 2173, 2176, 2177, 2181,
       2192, 2194, 2196, 2198, 2206, 2212, 2221, 2223, 2225, 2227, 2228,
       2234, 2241, 2251, 2261, 2276, 2282, 2291, 2302, 2304, 2306, 2308,
       2316, 2323, 2326, 2328, 2336, 2348, 2356, 2362, 2378, 2383, 2385,
       2387, 2392, 2395, 2398, 2402, 2404, 2406, 2413, 2414, 2417, 2424,
       2426, 2428, 2433, 2442, 2444, 2446, 2448, 2452, 2462, 2464, 2466,
       2468, 2477, 2483, 2484, 2488, 2498, 2504, 2518, 2522, 2538, 2541,
       2552, 2568, 2578, 2584, 2592, 2607, 2616, 2621, 2634, 2641, 2642,
       2646, 2658, 2662, 2672, 2675, 2676, 2682, 2683, 2686, 2687, 2691,
       2693, 2695, 2697, 2702, 2703, 2705, 2707, 27

In [4]:
df.Kategorie.unique()

array(['DTV (Kfz/24h)', 'DTVw (Kfz/24h)', 'SV-Anteil am DTVw (%)',
       'Baustelleneinfluss'], dtype=object)

In [5]:
# remove this category, translated "Construction site influence"
df = df[df['Kategorie'] != 'Baustelleneinfluss']

In [6]:
df

,Zählstelle,Ebene,Bezeichnung,Kategorie,2004,2005,2006,2007,2008,2009,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1013,371,Krohnstieg NO Garstedter Weg,DTV (Kfz/24h),51000,52000,52000,53000,52000,52000,...,53000,51000,51000,52000,51000,51000,42000,39000,47000,48000
1,1013,371,Krohnstieg NO Garstedter Weg,DTVw (Kfz/24h),56000,56000,57000,58000,57000,56000,...,57000,56000,56000,56000,56000,56000,46000,43000,50000,52000
2,1013,371,Krohnstieg NO Garstedter Weg,SV-Anteil am DTVw (%),6,6,6,6,6,6,...,5,5,5,4,5,5,5,6,5,5
4,1017,372,Garstedter Weg SW Krohnstieg,DTV (Kfz/24h),12000,11000,11000,11000,11000,11000,...,12000,12000,13000,12000,12000,12000,10000,9000,10000,9000
5,1017,372,Garstedter Weg SW Krohnstieg,DTVw (Kfz/24h),13000,13000,13000,13000,13000,12000,...,13000,14000,15000,14000,14000,14000,12000,11000,11000,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,7459,221,Hammer Straße NW Grenzknick,DTVw (Kfz/24h),18000,18000,19000,21000,21000,21000,...,20000,21000,21000,20000,22000,20000,23000,20000,23000,23000
1422,7459,221,Hammer Straße NW Grenzknick,SV-Anteil am DTVw (%),4,3,4,4,4,4,...,4,4,3,3,3,3,4,NaN,NaN,NaN
1424,7621,96,Eulenkrugstraße W Im Meienthun (Landesgrenze),DTV (Kfz/24h),11000,11000,11000,11000,12000,11000,...,13000,12000,12000,12000,12000,13000,10000,11000,11000,NaN
1425,7621,96,Eulenkrugstraße W Im Meienthun (Landesgrenze),DTVw (Kfz/24h),12000,12000,12000,12000,13000,12000,...,14000,14000,13000,13000,13000,15000,11000,13000,12000,NaN


In [7]:
# Reshape the DataFrame
df = pd.melt(
    df,
    id_vars=['Zählstelle', 'Ebene', 'Bezeichnung', 'Kategorie'],  # Columns to keep
    value_vars=[str(year) for year in range(2004, 2024)],  # Year columns
    var_name='year',  # New column name for years
    value_name='value'  # New column name for values
).pivot( # Now we pivot for the categories
    index=['Zählstelle', 'year','Ebene', 'Bezeichnung'], 
    columns='Kategorie', 
    values='value'
    # remove rows without values and rename columns
).dropna(how = 'all').reset_index().rename(columns = {
    'DTV (Kfz/24h)' : 'AADT',
    'DTVw (Kfz/24h)' : 'AAWT',
    'SV-Anteil am DTVw (%)' : 'TR_pct_AAWT',
     'Bezeichnung':'Street_name'
})

df

Kategorie,Zählstelle,year,Ebene,Street_name,AADT,AAWT,TR_pct_AAWT
0,1013,2004,371,Krohnstieg NO Garstedter Weg,51000,56000,6
1,1013,2005,371,Krohnstieg NO Garstedter Weg,52000,56000,6
2,1013,2006,371,Krohnstieg NO Garstedter Weg,52000,57000,6
3,1013,2007,371,Krohnstieg NO Garstedter Weg,53000,58000,6
4,1013,2008,371,Krohnstieg NO Garstedter Weg,52000,57000,6
...,...,...,...,...,...,...,...
6347,7621,2018,96,Eulenkrugstraße W Im Meienthun (Landesgrenze),12000,13000,2
6348,7621,2019,96,Eulenkrugstraße W Im Meienthun (Landesgrenze),13000,15000,2
6349,7621,2020,96,Eulenkrugstraße W Im Meienthun (Landesgrenze),10000,11000,2
6350,7621,2021,96,Eulenkrugstraße W Im Meienthun (Landesgrenze),11000,13000,NaN


# geographical reference

In [8]:
filenames = [
    'app_kfz_bedarfszaehlstellen_EPSG_4326',
    'app_kfz_dauerzaehlstellen_EPSG_4326',
    'app_kfz_jaehrliche_zaehlstellen_EPSG_4326',
    'app_kfz_pegel_EPSG_4326',
]

# demand counting points
# permanent counting points
# annual counting points
# level

# the most matching referential seems to be the last one

for f in filenames:
    print(gpd.read_file(f'raw/{f}.json')['zaehlstelle'].unique())


[   2    3    5 ... 7792 7793 7794]
[101 102 103 105 106 107 108 109 110 111 112 113 114 115 116 117 119 121
 122 258 354 355 371 376]
[104 118 120 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251
 252 253 254 255 256 257 259 260 261 262 263 264 265 266 267 268 269 270
 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306
 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342
 343 344 345 346 347 348 349 350 351 352 353 356 357 358 359 360 361 362
 363 364 365 366 367 368 369 370 372 373 374 375 377 378 379 380 381 382
 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400
 401 402 403 404 405 406 407 408 409 410]
[1013 1017 1018 1022

In [9]:
gdf = gpd.read_file(f'raw/{filenames[-1]}.json')


In [10]:
gdf

,id,zaehlstelle,bezeichnung,letzte_zaehlung,geometry
0,APP_KFZ_PEGEL_37025,1013,Krohnstieg NO Garstedter Weg T371 DP3233,30.06.2024,POINT (9.97557 53.64579)
1,APP_KFZ_PEGEL_37026,1017,Garstedter Weg SW Krohnstieg T372 DP3334,30.06.2024,POINT (9.97424 53.64494)
2,APP_KFZ_PEGEL_37027,1018,Swebenweg W Krohnstieg T373 DP3234,30.06.2024,POINT (9.97385 53.64546)
3,APP_KFZ_PEGEL_37028,1022,Zeppelinstraße N Terminalzufahrten (UFu) T367 ...,30.06.2024,POINT (10.00963 53.63582)
4,APP_KFZ_PEGEL_37029,1035,Kollaustraße SO Vogt-Cordes-Damm T2 DP01,30.06.2024,POINT (9.95257 53.61327)
...,...,...,...,...,...
351,APP_KFZ_PEGEL_37376,4088,Am Sandtorkai W Bei St. Annen T406,04.01.2016,POINT (9.99741 53.54401)
352,APP_KFZ_PEGEL_37377,4092,Osakaallee N Überseeallee T407,08.10.2015,POINT (9.9996 53.54146)
353,APP_KFZ_PEGEL_37378,4094,Überseeallee O Osakaallee T408,08.10.2015,POINT (10.0001 53.54116)
354,APP_KFZ_PEGEL_37379,4098,Überseeallee W Osakaallee T409,08.10.2015,POINT (9.99927 53.54125)


# Map matching with OSM

In [11]:
import sys
from pathlib import Path

# Add the ../assets directory to sys.path
sys.path.append(str(Path("../../assets").resolve()))

from map_matching_OSM import points_matching

In [12]:
gdf = points_matching(gdf)

City downloaded


 36%|███▌      | 129/356 [00:05<00:07, 29.13it/s]

Value Error - No roads found nearby current index


 63%|██████▎   | 225/356 [00:09<00:06, 19.36it/s]

Value Error - No roads found nearby current index


 70%|██████▉   | 249/356 [00:10<00:04, 21.90it/s]

Value Error - No roads found nearby current index


100%|██████████| 356/356 [00:14<00:00, 24.15it/s]

We failed to match 3 sensors
...on a total of 356 sensors


In [13]:
gdf[['bezeichnung', 'osm_name']].drop_duplicates()[:20]

,bezeichnung,osm_name
0,Krohnstieg NO Garstedter Weg T371 DP3233,Krohnstieg
1,Garstedter Weg SW Krohnstieg T372 DP3334,Garstedter Weg
2,Swebenweg W Krohnstieg T373 DP3234,Swebenweg
3,Zeppelinstraße N Terminalzufahrten (UFu) T367 ...,Zeppelinstraße
4,Kollaustraße SO Vogt-Cordes-Damm T2 DP01,Kollaustraße
5,A23 NW AD Hamburg-Nordwest T344 DP2930,NaN
6,A7 NO AD Hamburg-Nordwest T343 DP2830,NaN
7,A7 SO AD Hamburg-Nordwest T342 DP2829,NaN
8,Luruper Hauptstraße O Engelbrechtweg (Landesgr...,Luruper Hauptstraße
9,Fruchtallee NW Emilienstraße T11 DP10,Fruchtallee


# Treatment

In [14]:
df.rename(columns = {'Zählstelle':'Street_id'}, inplace=True)
gdf.rename(columns = {'zaehlstelle':'Street_id'}, inplace=True)

In [15]:
df = df.set_index('Street_id').join(
    gdf.set_index('Street_id'),
    how = 'inner'
)

In [16]:
df.shape

(6237, 17)

In [17]:
df.isna().sum()

year                  0
Ebene                 0
Street_name           0
AADT                  0
AAWT                  0
TR_pct_AAWT         640
id                    0
bezeichnung           0
letzte_zaehlung       0
geometry              0
osm_name            856
osm_type             57
osm_lanes           243
osm_oneway         2430
osm_distance         57
osm_maxspeed        390
osmid                57
dtype: int64

In [18]:
for k in ['AADT', 'AAWT', 'TR_pct_AAWT']:
    df[k] = df[k].astype(float)

In [19]:
df = gpd.GeoDataFrame(
    df,
    geometry = 'geometry',
    crs = 'epsg:4326'
)

In [20]:
df.columns

Index(['year', 'Ebene', 'Street_name', 'AADT', 'AAWT', 'TR_pct_AAWT', 'id',
       'bezeichnung', 'letzte_zaehlung', 'geometry', 'osm_name', 'osm_type',
       'osm_lanes', 'osm_oneway', 'osm_distance', 'osm_maxspeed', 'osmid'],
      dtype='object')

In [21]:
df.rename(columns = {'Street_name':'raw_name'}, inplace=True)

In [24]:
from validate_dataset import validate_dataset

df = validate_dataset(df)

Number of NaN values for AADT: 0
Number of NaN values for geometry: 0
Number of low outliers for AADT: 0
Number of high outliers for AADT: 0


In [25]:
# Final save

for year in range(2015, 2024):
    print(year, df[df.year == str(year)].shape[0])
    df[df.year == str(year)][[
        'AADT', 'AAWT', 'TR_pct_AAWT', 'geometry', 'raw_name', 'osm_name', 'osm_type', 'osm_lanes', 'osm_oneway',
       'osm_distance', 'osm_maxspeed', 'osmid'
        ]].to_file(
        f'treated/Hamburg_AADT_AAWT_{year}.geojson', index=False
    )

2015 329
2016 346
2017 347
2018 348
2019 347
2020 295
2021 257
2022 264
2023 208
